In [2]:
import pandas as pd
import numpy as np

### Read metrics csv

In [3]:
author_cnt = pd.read_csv('author_cnt.csv',header=None)
id_cnt = pd.read_csv('id_cnt.csv',header=None)
subscribers_avg = pd.read_csv('subscribers_avg.csv',header=None)
score_avg = pd.read_csv('score_avg.csv',header=None)
comment_avg = pd.read_csv('comment_avg.csv',header=None)
retention_rate = pd.read_csv('retention_rate.csv')

In [75]:
def rename(dataset):
    dataset.loc[dataset['subreddit'] == '2th52','subreddit'] = 'wallstreetbets'
    dataset.loc[dataset['subreddit'] == '2s7v0','subreddit'] = 'SecurityAnalysis'
    dataset.loc[dataset['subreddit'] == '2qhhq','subreddit'] = 'investing'
    dataset.loc[dataset['subreddit'] == '2qjfk','subreddit'] = 'stocks'
    dataset.loc[dataset['subreddit'] == '2uud8','subreddit'] = 'RobinHood'

In [68]:
df_list = [author_cnt,id_cnt,subscribers_avg,score_avg,comment_avg]
name_list = ['author_cnt','id_cnt','subscribers_avg','score_avg','comment_avg']
for i in range(len(df_list)):
    df_list[i].columns =[name_list[i],'subreddit','date']
    df_list[i]['date'] = pd.to_datetime(df_list[i].date, format='%m/%d/%y')
    rename(df_list[i])

In [69]:
reddit_list = [
'wallstreetbets',
'SecurityAnalysis',
'investing',
'stocks',
'RobinHood']

In [70]:
date = pd.date_range('2019-01-01','2021-02-28')
df_whole = pd.DataFrame({ 'date': date }) 

### Combine dataframes to one

In [71]:
for i in range(len(df_list)):
    grouped = df_list[i].groupby(['subreddit'])
    for reddit in reddit_list:
        df = grouped.get_group(reddit)
        df = df.drop('subreddit',axis=1)
        df.rename(columns={name_list[i]: reddit+'_'+name_list[i]}, inplace=True)
        df_whole = df_whole.merge(df, on='date', how='outer')

In [83]:
rename(retention_rate)
retention_rate['date'] = pd.to_datetime(retention_rate.date, format='%Y/%m/%d')

grouped = retention_rate.groupby(['subreddit'])
for reddit in reddit_list:
    df = grouped.get_group(reddit)
    df = df.drop('subreddit',axis=1)
    df.rename(columns={'retention_rate': reddit+'_'+'retention_rate'}, inplace=True)
    df_whole = df_whole.merge(df, on='date', how='outer')

In [85]:
connectivity = pd.read_csv('connectivity.csv',parse_dates=['date'])
df_whole = df_whole.merge(connectivity, on='date', how='outer')

### Output final csv

In [90]:
df_whole.to_csv('network_metrics_daily.csv',index=False)